In [72]:
 # Dependencies and Setup
import pandas as pd

In [11]:
# Files to Load
student_input = "Resources/students_complete.csv"
school_input = "Resources/schools_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
student_data = pd.read_csv(student_input)
school_data = pd.read_csv(school_input)

# Combine the data into a single dataset.  
df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

**Create a high-level snapshot, in a DataFrame, of the district's key metrics, including the following:**

- Total schools
- Total students
- Total budget
- Average math score
- Average reading score
- % passing math (the percentage of students who passed math)
- % passing reading (the percentage of students who passed reading)
- % overall passing (the percentage of students who passed math AND reading)

In [128]:
# create dataframe to show total with columns for schools, students, and budget
test_dict = {"Total Schools" : df["School ID"].nunique(), "Total Students" : df["Student ID"].nunique()}
test_dict

{'Total Schools': 15, 'Total Students': 39170}

ValueError: Length mismatch: Expected 39170 rows, received array of length 15

In [94]:
# create dataframe to show avg with columns math score and reading score
avg_df = df.loc[:,["math_score","reading_score"]].mean()
avg_df.head()

In [116]:
# create new columns that show % passing for math, reading, and combined
newDF = df
newDF["% passing math"] = (df["math_score"].mean() / df["student_name"].nunique()) * 100
newDF.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,% passing math
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,0.241435
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,0.241435
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,0.241435
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,0.241435
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,0.241435


## School Summary

**Create a DataFrame that summarizes key metrics about each school, including the following:**

- School name
- School type
- Total students
- Total school budget
- Per student budget
- Average math score
- Average reading score
- % passing math (the percentage of students who passed math)
- % passing reading (the percentage of students who passed reading)
- % overall passing (the percentage of students who passed math AND reading)

## Highest-Performing Schools (by % Overall Passing)

Create a DataFrame that highlights the top 5 performing schools based on % Overall Passing. Include the following metrics:

* School name
* School type
* Total students
* Total school budget
* Per student budget
* Average math score
* Average reading score
* % passing math (the percentage of students who passed math)
* % passing reading (the percentage of students who passed reading)
* % overall passing (the percentage of students who passed math AND reading)

## Lowest-Performing Schools (by % Overall Passing)

Create a DataFrame that highlights the bottom 5 performing schools based on % Overall Passing. Include the following metrics:

* School name
* School type
* Total students
* Total school budget
* Per student budget
* Average math score
* Average reading score
* % passing math (the percentage of students who passed math)
* % passing reading (the percentage of students who passed reading)
* % overall passing (the percentage of students who passed math AND reading)

## Math Scores by Grade

Create a DataFrame that lists the average math score for students of each grade level (9th, 10th, 11th, 12th) at each school.

## Reading Scores by Grade

Create a DataFrame that lists the average reading score for students of each grade level (9th, 10th, 11th, 12th) at each school.

## Scores by School Spending

Create a table that breaks down school performance based on average spending ranges (per student). Use your judgment to create four bins with reasonable cutoff values to group school spending. Include the following metrics in the table:

* Average math score
* Average reading score
* % passing math (the percentage of students who passed math)
* % passing reading (the percentage of students who passed reading)
* % overall passing (the percentage of students who passed math AND reading)

## Scores by School Size

Create a table that breaks down school performance based on school size (small, medium, or large).

## Scores by School Type

Create a table that breaks down school performance based on type of school (district or charter).